# Duration Prediction

In [40]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [41]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')

In [42]:
# df = df.loc[(df.trip_type == 2), :]

In [43]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [44]:
df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]

In [45]:
df.duration.describe(percentiles=[.95, .98, .99])

count    73908.000000
mean        16.852578
std         11.563163
min          1.000000
50%         14.000000
95%         41.000000
98%         48.781000
99%         53.000000
max         60.000000
Name: duration, dtype: float64

In [53]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [54]:
def one_hot_encoding(df, numerical, categorical, dv=None):
    df[categorical] = df[categorical].astype(str)
    train_dicts = df[numerical + categorical].to_dict(orient='records')
    if not dv:
        dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts)

    return dv, X_train

In [56]:
_, X_train = one_hot_encoding(df, numerical, categorical)

In [57]:
X_train

<73908x507 sparse matrix of type '<class 'numpy.float64'>'
	with 221724 stored elements in Compressed Sparse Row format>

In [47]:
target = 'duration'
y_train = df[target].values

In [ ]:
lr = LinearRegression()